In [ ]:
!pip install ir_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18920 sha256=8977368b05a65f7982987efcc7c4d63141a3301d3c1662bf09f1d8b5f8088045
  Stored in directory: /root/.cache/pip/wheels/7f/22/ed/a11944d7fdf4e94c4206a3f760d385122a4d34d8acc12f71a3
  Created wheel for cbor: filename=cbor-1.0.0-cp39-cp39-linux_x86_64.whl size=57282 sha256=7585b27bb3399d79d722793ee4d755dda221413bd84

In [ ]:
import ir_datasets
dataset = ir_datasets.load('cranfield')

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download necessary nltk resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
stopwords.words("english")[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """Function that cleans the input text by going to:
    - remove links
    - remove special characters
    - remove numbers
    - remove stopwords
    - convert to lowercase
    - remove excessive white spaces
    Arguments:
        text (str): text to clean
        remove_stopwords (bool): whether to remove stopwords
    Returns:
        str: cleaned text
    """
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    text = text.lower()
    # remove links
    #text = re.sub(r"http\S+", "", text)
    # remove numbers and special characters
    #text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. create tokens
        tokens = nltk.word_tokenize(text)
        # 2. check if it's a stopword
        tokens = [w.lower().strip() for w in tokens if not w.lower() in stopwords.words("english")]
        # return a list of cleaned tokens
        return tokens

In [ ]:
def preprocess_text(text, remove_stopwords: bool):
    # Convert text to lower case
    text = text.lower()
    text = ''.join(c for c in text if c.isalnum() or c.isspace())

    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Stem tokens
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]

    # Join tokens back into a single string
    #preprocessed_text = ' '.join(stemmed_tokens)

    return stemmed_tokens

In [ ]:
import pandas as pd
df_docs = pd.DataFrame(columns=['doc_id', 'processed_text'])
df_q = pd.DataFrame(columns=['q_id', 'processed_text'])
for item in dataset.docs_iter():
  proc1 = preprocess_text(item[1], remove_stopwords=True)
  proc2 = preprocess_text(item[2], remove_stopwords=True)
  #df.loc[len(df.index)] = [item[0], [preprocess_text(item[1], remove_stopwords=True), preprocess_text(item[2], remove_stopwords=True)]] 
  proc1.extend(proc2)
  df_docs.loc[len(df_docs.index)] = [item[0], proc1]
for item in dataset.queries_iter():
  df_q.loc[len(df_q.index)] = [item[0], preprocess_text(item[1], remove_stopwords=True)] 
result = df_q.head(10)
print("First 10 rows of the DataFrame:")
print(result)

[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
[INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz: [00:00] [507kB] [4.08MB/s]


First 10 rows of the DataFrame:
  q_id                                     processed_text
0    1  [similar, law, must, obey, construct, aeroelas...
1    2  [structur, aeroelast, problem, associ, flight,...
2    4  [problem, heat, conduct, composit, slab, solv,...
3    8  [criterion, develop, show, empir, valid, flow,...
4    9  [chemic, kinet, system, applic, hyperson, aero...
5   10  [theoret, experiment, guid, turbul, couett, fl...
6   12  [possibl, relat, avail, pressur, distribut, og...
7   13  [method, dash, exact, approxim, dash, present,...
8   15  [paper, intern, slip, flow, heat, transfer, st...
9   18  [realga, transport, properti, air, avail, wide...


# **Section 3**

In [ ]:
from gensim.models import Word2Vec



texts1 = df_docs.processed_text.tolist()
texts2 = df_q.processed_text.tolist()
texts1.extend(texts2)

wvmodel = Word2Vec(
        texts1,
        size=100,
        window=10,
        min_count=1,
        workers=10,
        iter=10)
wvmodel.save("word2vec.model")


### glove

In [ ]:
# coding: utf-8
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


# Convert
input_file = 'glove.6B.100d.txt'
output_file = 'gensim_glove.6B.100d.txt'
glove2word2vec(input_file, output_file)


# Test Glove model
model = KeyedVectors.load_word2vec_format(output_file, binary=False)
word = 'cat'
print(word)
#print('Most similar:\n{}'.format(model.most_similar(word)))


FileNotFoundError: ignored

In [ ]:
model.get_vector('cat')

array([ 0.23088  ,  0.28283  ,  0.6318   , -0.59411  , -0.58599  ,
        0.63255  ,  0.24402  , -0.14108  ,  0.060815 , -0.7898   ,
       -0.29102  ,  0.14287  ,  0.72274  ,  0.20428  ,  0.1407   ,
        0.98757  ,  0.52533  ,  0.097456 ,  0.8822   ,  0.51221  ,
        0.40204  ,  0.21169  , -0.013109 , -0.71616  ,  0.55387  ,
        1.1452   , -0.88044  , -0.50216  , -0.22814  ,  0.023885 ,
        0.1072   ,  0.083739 ,  0.55015  ,  0.58479  ,  0.75816  ,
        0.45706  , -0.28001  ,  0.25225  ,  0.68965  , -0.60972  ,
        0.19578  ,  0.044209 , -0.31136  , -0.68826  , -0.22721  ,
        0.46185  , -0.77162  ,  0.10208  ,  0.55636  ,  0.067417 ,
       -0.57207  ,  0.23735  ,  0.4717   ,  0.82765  , -0.29263  ,
       -1.3422   , -0.099277 ,  0.28139  ,  0.41604  ,  0.10583  ,
        0.62203  ,  0.89496  , -0.23446  ,  0.51349  ,  0.99379  ,
        1.1846   , -0.16364  ,  0.20653  ,  0.73854  ,  0.24059  ,
       -0.96473  ,  0.13481  , -0.0072484,  0.33016  , -0.1236

## **section 4**

In [ ]:
pip install sister

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install 'sister[bert]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sister
#import gensim

# Create pre-trained Word2Vec model using SISTER library
#word_embedding = sister.MeanEmbedding(lang="en")

# Load pre-trained BERT model using SISTER library
bert_embedding = sister.BertEmbedding(lang="en")

# Save pre-trained Word2Vec model
#word_embedding.save("word2vec.model")

# Save pre-trained BERT model
bert_model = bert_embedding.model
#bert_model.save("bert.model")

ImportError: ignored

In [ ]:
sentences = ["I am a dog.", "I want be a cat."]
vectors = bert_embedding(sentences)
print(vectors)

NameError: ignored

## **section 6**

In [ ]:
import numpy as np
def get_embedding(query,model):
  #print(query)
  wordList = query.split()
  #print(wordList)
  total = np.zeros(100)
  for word in wordList:
    total = total + model.wv[word]
  average = total / len(wordList)
  return average


In [ ]:
x = get_embedding("theory impact tube",wvmodel)
print(x)

[ 0.3118125   0.05219336 -0.44801031  0.29703744  0.33513015 -0.26586321
 -0.05281793 -0.09016544  0.21458884 -0.21530903 -0.22829014  0.02760844
  0.17243161  0.12068731  0.05064794  0.06505155  0.18894126  0.13246674
  0.17036896  0.15415555  0.11618797 -0.05810861  0.058906    0.164241
 -0.52243017  0.02144179 -0.21433885 -0.12336089 -0.01644604 -0.11843145
 -0.52415323 -0.08789347  0.06871861  0.34887805 -0.18157338  0.18085666
  0.09050591 -0.36506294 -0.11037011  0.62739014 -0.11605056  0.22868201
  0.42224648 -0.03824583 -0.07632057  0.03317567 -0.09764169 -0.4057217
  0.08153861 -0.03576794  0.12663705 -0.44305834  0.26740193  0.05377997
 -0.09671239 -0.11099678  0.09120347 -0.18712352  0.05500849  0.20355696
  0.02352884  0.02807646  0.2502099   0.48500536 -0.14380973 -0.45602903
 -0.38680805 -0.10616841 -0.11620299 -0.28922119 -0.1447325  -0.48934894
 -0.18574931 -0.10388587  0.03742166 -0.23356482  0.21799789 -0.06017007
 -0.13487025  0.05215491 -0.29305512 -0.0358076  -0.10

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_score(document,query,model):
  query_emb = get_embedding(query,model)
  document_emb = get_embedding(document,model)
  similarity = cosine_similarity(query_emb.reshape(1, -1), document_emb.reshape(1, -1))
  return similarity


In [ ]:
print(get_score("theory impact tube","theory",wvmodel))

[[0.90125808]]


In [ ]:
def calculateAllScores(model):
  df_qrel = pd.DataFrame(columns=['query_id','doc_id','score','relevance'])
  for item in dataset.qrels_iter():
    df_qrel.loc[len(df_qrel.index)] = [item[0], item[1], 0,item[2]] 
  
  for item in df_qrel.iterrows():
    try:
     s_doc = ' '.join((((df_docs.loc[df_docs['doc_id'] == item[1].doc_id]).processed_text).tolist())[0])
     s_query = ' '.join((((df_q.loc[df_q['q_id'] == item[1].query_id]).processed_text).tolist())[0])
     item[1].score = get_score(s_doc,s_query,model)[0][0]
     #df_qrel.at[item.index,'score']=get_score(s_doc,s_query,model)[0][0]
     #print(df_qrel.at[item.index,'score'])
     print(item[1].score)
    except:
     print("exception")
     continue
  return df_qrel

#df_qrel.head(200)


In [ ]:
def sortScores(model):
  df_qrel = calculateAllScores(model)
  df_qrel['query_id'] = pd.to_numeric(df_qrel['query_id'])
  df_qrel['doc_id'] = pd.to_numeric(df_qrel['doc_id'])
  
  
  df_sorted = df_qrel.sort_values(by=['query_id', 'score'], ascending=[True, False])
  return df_sorted

In [ ]:
import math

def DCG(model):
  df_sorted = sortScores(model)
  unique_queries = df_sorted['query_id'].unique()
  original_queries = (pd.to_numeric(df_q['q_id'])).unique()
  listOfQueries = list(set(unique_queries) & set(original_queries))
  #print(unique_queries)
  mean_DCG = 0
  for item in listOfQueries:
    docs = df_sorted.loc[df_sorted['query_id'] == item]
    #print(docs)
    DCG = 0
    k = len(docs)
    for i in range(1,k+1):
      DCG += ((2 ** (float(docs.iloc[i-1].relevance))) / math.log(i+1, 2))
    mean_DCG += DCG
  mean_DCG /= len(unique_queries)
  #print(mean_DCG)
  return (df_sorted)




In [ ]:
#print("DCG for pretrained Word2Vec: ",DCG(wvmodel))
DCG(wvmodel)
#calculateAllScores(wvmodel)

0.811855533910166
0.6143135627053746
0.5110770810958396
0.8682809395248352
0.813495477118264
0.8978713992526944
0.5904140441791996
0.725212118755941
0.5035717180770406
0.5689737092220521
0.6963334062806598
0.6515991264233523
0.8270885572131668
0.6804439168202517
0.7245085801283928
0.698857769336408
0.7012401485603293
0.6094076420126958
0.7679626927005458
0.570127975575375
0.5957279677784629
0.714622865933573
0.6976800473994917
0.7021329262083598
0.6238833401175657
0.8470156602891274
0.6836635070802124
0.7354988564903615
0.817799949209397
0.8751574817471812
0.4174690832479591
0.6214354801953965
0.4900407120335708
0.7830911341786383
0.7400955184803771
0.7701835187190724
0.5911700158544456
0.6554421882448898
0.452920683906462
0.8166176675320223
0.5260477985190734
0.5735849131773525
0.6239668310635401
0.6267150703029688
0.6887167161982419
0.5295754207390698
0.5061282936761855
0.6891930197686126
0.583620420177337
0.45484512057722853
0.653841387766166
0.4629257961754127
0.8121798271569761
0.

,query_id,doc_id,score,relevance
0,1,184,0,2
1,1,29,0,2
2,1,31,0,2
3,1,12,0,3
4,1,51,0,3
...,...,...,...,...
1832,225,1062,0,3
1833,225,1074,0,3
1834,225,1075,0,3
1835,225,1213,0,3
